# **Project Overview:**

This project aims to develop a machine learning model for estimating the appropriate dose of the anesthetic drug propofol for patients undergoing medical procedures. We have an initial dataset from a hospital containing patient information and anesthesia records. This data includes:

    Demographics: Gender (1=Male, 2=Female), Age, Height, Weight
    Physiological Measures: Systolic and Diastolic Blood Pressure, Heart Rate
    Anesthesia Details: Depth of Anesthesia, Actual Dose of Propofol Administered

## **Data Augmentation and Model Training:**

To enhance the model's performance, we will generate 300 synthetic data points that mimic real patient data. These synthetic data points will have "true" propofol doses assigned through the simulation process. We will then combine these synthetic data points with the original 13 data points to train a machine learning model. The goal is for the model to accurately estimate the propofol dose for new, unseen patients, with the estimated doses closely approximating the "true" doses in the synthetic data.

## **Additional Data for Reference:**

The following Python dictionary (base_data) provides a structured view of the initial 13 data points, which can be helpful for understanding the data format and features.

## **Addressing Dimensionality Challenges in the Dataset**

A key challenge in this dataset is the varying dimensionality of different features. For instance, we have features like age, gender, height, and weight, which are primarily scalar (one-dimensional). However, alongside these, we encounter features like heart rhythm, systolic and diastolic blood pressure, and depth of anesthesia. These are vector quantities, each represented as a list of multiple measurements or values.

## **Data Transformation and Model Training**

To address this dimensionality mismatch, we transform the dataset. We create multiple datasets, each corresponding to a specific induction step. The size of each dataset aligns with the dimensionality of the vector-valued features. We then train a single, unified machine learning model on these transformed datasets. This same model is subsequently used for making predictions on new data.



### import used library`s

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

### Splitting pairs of systolic and diastolic blood pressure data into separate data

In [2]:
blood_p =[[[57, 115], [57, 115], [57, 115], [57, 115], [57, 115], [65, 111], [65, 111]],
        [[60, 136], [60, 136], [60, 136], [60, 136], [60, 136], [78, 121], [78, 121]],
        [[56, 126], [56, 126], [56, 126], [56, 126], [56, 126], [69, 111], [69, 111]],
        [[79, 150], [79, 150], [79, 150], [79, 150], [79, 150], [74, 144], [74, 144]],
        [[62, 146], [62, 146], [62, 146], [62, 146], [62, 146], [96, 159], [96, 159]],
        [[80, 120], [80, 120], [80, 120], [80, 120], [80, 120], [71, 116], [71, 116]],
        [[78, 115], [78, 115], [78, 115], [78, 115], [78, 115], [69, 102], [69, 102]],
        [[70, 110], [70, 110], [70, 110], [70, 110], [70, 110], [78, 120], [78, 120]],
        [[88, 138], [88, 138], [88, 138], [88, 138], [88, 138], [70, 111], [70, 111]],
        [[60, 131], [60, 131], [60, 131], [60, 131], [60, 131], [55, 110], [55, 110]],
        [[75, 115], [75, 115], [75, 115], [75, 115], [75, 115], [61, 101], [61, 101]],
        [[87, 120], [87, 120], [87, 120], [87, 120], [87, 120], [80, 115], [80, 115]],
        [[66, 129], [66, 129], [66, 129], [66, 129], [66, 129], [70, 135], [70, 135]]]

systolic = []
diastolic = []
syst_min = 180
syst_max = 0
diast_min = 250
diast_max = 0

for i in blood_p : 
    syst = []
    diast = []
    for j in i :
        for z in range(len(j)) : 
            if z % 2 == 0 :
                syst.append(j[z])
                if j[z] > syst_max :
                    syst_max = j[z]
                if j[z] < syst_min :
                    syst_min = j[z]
            else :
                diast.append(j[z])
                if j[z] > diast_max :
                    diast_max = j[z]
                if j[z] < diast_min :
                    diast_min = j[z]
    systolic.append(syst)
    diastolic.append(diast)

### Forming a data dictionary using the given data

In [3]:
base_data = {'sick_number': [1,2,3,4,5,6,7,8,9,10,11,12,13],
            'sick_gender': [1,1,1,2,1,2,1,1,1,2,1,1,2],
            'sick_age': [40,36,50,28,43,37,38,41,37,42,34,28,38],
            'sick_height': [163,163,163,164,163,187,174,170,167,179,172,164,169],
            'sick_weight': [54,50,83,60,59,75,80,79,58,78,58,52,65],
            'sick_blood_pressure_systolic': systolic,
            'sick_blood_pressure_diastolic': diastolic,
            'sick_heart_rhythm':[[127,127,127,127,127,101,101],[102,102,102,102,102,74,74],[100,100,100,100,100,97,97],[98,98,98,98,98,82,82],[102,102,102,102,102,80,80],[140,140,140,140,140,162,162],[112,112,112,112,112,101,101],[110,110,110,110,110,87,87],[90,90,90,90,90,77,77],[123,123,123,123,123,114,114],[105,105,105,105,105,85,85],[134,134,134,134,134,99,99],[119,119,119,119,119,94,94]],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'induction_steps': [[i for i in range(7)] for j in range(13)],
            'depth_of_patient_anesthesia':[[99,98,90,61,48,49,50],[99,95,88,65,51,50,50],[97,94,70,56,52,51,50],[97,96,88,63,51,50,50],[95,91,68,54,51,50,50],[93,92,80,60,51,50,50],[92,89,78,64,58,55,54],[89,89,86,56,47,51,50],[83,80,71,65,59,57,55],[92,85,64,53,51,50,50],[90,89,80,68,59,55,52],[97,88,62,53,51,50,50],[100,93,90,70,65,56,50]]}



### Formation of correlation matrix by pre-processing inconsistent data and removing irrelevant data such as patient number

In [4]:
corr_data = base_data.copy()
corr_data['depth_of_patient_anesthesia'] = [np.mean(i) for i in base_data['depth_of_patient_anesthesia']]
corr_data['sick_blood_pressure_systolic'] = [np.mean(i) for i in base_data['sick_blood_pressure_systolic']]
corr_data['sick_blood_pressure_diastolic'] = [np.mean(i) for i in base_data['sick_blood_pressure_diastolic']]
corr_data['sick_heart_rhythm'] = [np.mean(i) for i in base_data['sick_heart_rhythm']]
main_df = pd.DataFrame(corr_data).drop(['sick_number','induction_steps'], axis=1)



### Drawing correlation matrix and pairwise diagrams to visualize and analyze multivariate relationships


#### Specifically, this function("pairplot") creates a matrix of graphs:

    Non-diagonal plots: show scatter plots between each pair of variables. These graphs help you examine the relationships between variables.
    Diagonal bars: show the distribution of each variable separately. A histogram or density chart is usually used for this purpose.

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(main_df.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()
sns.pairplot(main_df, diag_kind='kde')
plt.show()

### Dividing data based on their induction stages for better and easier pre-processing and converting vector data such as lists into single numerical data, as well as the attention of our machine learning model to the separation of these stages

In [6]:
induction_steps_0 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][0] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][0] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][0] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][0] for i in range(len(base_data['depth_of_patient_anesthesia']))]}



induction_steps_1 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][1] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][1] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][1] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][1] for i in range(len(base_data['depth_of_patient_anesthesia']))]}



induction_steps_2 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][2] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][2] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][2] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][2] for i in range(len(base_data['depth_of_patient_anesthesia']))]}



induction_steps_3 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][3] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][3] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][3] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][3] for i in range(len(base_data['depth_of_patient_anesthesia']))]}



induction_steps_4 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][4] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][4] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][4] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][4] for i in range(len(base_data['depth_of_patient_anesthesia']))]}



induction_steps_5 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][5] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][5] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][5] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][5] for i in range(len(base_data['depth_of_patient_anesthesia']))]}



induction_steps_6 = {'sick_number': [i for i in base_data['sick_number']],
            'sick_gender': [i for i in base_data['sick_gender']],
            'sick_age': [i for i in base_data['sick_age']],
            'sick_height': [i for i in base_data['sick_height']],
            'sick_weight': [i for i in base_data['sick_weight']],
            'sick_blood_pressure_systolic': [base_data['sick_blood_pressure_systolic'][i][6] for i in range(len(base_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [base_data['sick_blood_pressure_diastolic'][i][6] for i in range(len(base_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[base_data['sick_heart_rhythm'][i][6] for i in range(len(base_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[2,2,2,1,2,1,2,2,2,1,2,2,2],
            'estimated_dose_of_propofol':[1.6,1.9,1.2,2,1.5,1.8,1.75,1.5,1.7,1.6,1.4,2,1.2],
            'depth_of_patient_anesthesia':[base_data['depth_of_patient_anesthesia'][i][6] for i in range(len(base_data['depth_of_patient_anesthesia']))]}

### Creating a data frame for each stage of induction and numbering it

In [7]:
df_0 = pd.DataFrame()
df_1 = pd.DataFrame()
df_2 = pd.DataFrame()
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()
df_5 = pd.DataFrame()
df_6 = pd.DataFrame()

for key, value in induction_steps_0.items():
    df_0[key + "_0"] = value
    
for key, value in induction_steps_1.items():
    df_1[key + "_1"] = value
    
for key, value in induction_steps_2.items():
    df_2[key + "_2"] = value
    
for key, value in induction_steps_3.items():
    df_3[key + "_3"] = value
    
for key, value in induction_steps_4.items():
    df_4[key + "_4"] = value
    
for key, value in induction_steps_5.items():
    df_5[key + "_5"] = value
    
for key, value in induction_steps_6.items():
    df_6[key + "_6"] = value

### Description of the data, including the number and quartiles, the average and standard deviation, and their minimum and maximum

In [ ]:
pd.DataFrame(base_data).describe() 

In [ ]:
df_0.describe()

### Data preprocessing and scaling with the standard scaler function to improve the training phase of the model and the insensitivity of the model to numerical distances.

In [10]:
d_o_p_a_scaling = []
for i in base_data['depth_of_patient_anesthesia'] : 
    for j in i : 
        d_o_p_a_scaling.append([j])
       
s_b_p_s_scaling = []
for i in base_data['sick_blood_pressure_systolic'] :
    for j in i : 
        s_b_p_s_scaling.append([j])
        
s_b_p_d_scaling = []
for i in base_data['sick_blood_pressure_diastolic'] :
    for j in i : 
        s_b_p_d_scaling.append([j])
        
s_h_r_scaling = []
for i in base_data['sick_heart_rhythm'] : 
    for j in i : 
        s_h_r_scaling.append([j])
        
d_o_p_a_scaler = StandardScaler()
d_o_p_a_scaler.fit(np.array(d_o_p_a_scaling))
df_0['depth_of_patient_anesthesia_0'] =  d_o_p_a_scaler.transform(df_0['depth_of_patient_anesthesia_0'].values.reshape(-1, 1))
df_1['depth_of_patient_anesthesia_1'] =  d_o_p_a_scaler.transform(df_1['depth_of_patient_anesthesia_1'].values.reshape(-1, 1))
df_2['depth_of_patient_anesthesia_2'] =  d_o_p_a_scaler.transform(df_2['depth_of_patient_anesthesia_2'].values.reshape(-1, 1))
df_3['depth_of_patient_anesthesia_3'] =  d_o_p_a_scaler.transform(df_3['depth_of_patient_anesthesia_3'].values.reshape(-1, 1))
df_4['depth_of_patient_anesthesia_4'] =  d_o_p_a_scaler.transform(df_4['depth_of_patient_anesthesia_4'].values.reshape(-1, 1))
df_5['depth_of_patient_anesthesia_5'] =  d_o_p_a_scaler.transform(df_5['depth_of_patient_anesthesia_5'].values.reshape(-1, 1))
df_6['depth_of_patient_anesthesia_6'] =  d_o_p_a_scaler.transform(df_6['depth_of_patient_anesthesia_6'].values.reshape(-1, 1))

s_b_p_s_scaler = StandardScaler()
s_b_p_s_scaler.fit(np.array(s_b_p_s_scaling))
df_0['sick_blood_pressure_systolic_0'] =  s_b_p_s_scaler.transform(df_0['sick_blood_pressure_systolic_0'].values.reshape(-1, 1))
df_1['sick_blood_pressure_systolic_1'] =  s_b_p_s_scaler.transform(df_1['sick_blood_pressure_systolic_1'].values.reshape(-1, 1))
df_2['sick_blood_pressure_systolic_2'] =  s_b_p_s_scaler.transform(df_2['sick_blood_pressure_systolic_2'].values.reshape(-1, 1))
df_3['sick_blood_pressure_systolic_3'] =  s_b_p_s_scaler.transform(df_3['sick_blood_pressure_systolic_3'].values.reshape(-1, 1))
df_4['sick_blood_pressure_systolic_4'] =  s_b_p_s_scaler.transform(df_4['sick_blood_pressure_systolic_4'].values.reshape(-1, 1))
df_5['sick_blood_pressure_systolic_5'] =  s_b_p_s_scaler.transform(df_5['sick_blood_pressure_systolic_5'].values.reshape(-1, 1))
df_6['sick_blood_pressure_systolic_6'] =  s_b_p_s_scaler.transform(df_6['sick_blood_pressure_systolic_6'].values.reshape(-1, 1))

s_b_p_d_scaler = StandardScaler()
s_b_p_d_scaler.fit(np.array(s_b_p_d_scaling))
df_0['sick_blood_pressure_diastolic_0'] =  s_b_p_d_scaler.transform(df_0['sick_blood_pressure_diastolic_0'].values.reshape(-1, 1))
df_1['sick_blood_pressure_diastolic_1'] =  s_b_p_d_scaler.transform(df_1['sick_blood_pressure_diastolic_1'].values.reshape(-1, 1))
df_2['sick_blood_pressure_diastolic_2'] =  s_b_p_d_scaler.transform(df_2['sick_blood_pressure_diastolic_2'].values.reshape(-1, 1))
df_3['sick_blood_pressure_diastolic_3'] =  s_b_p_d_scaler.transform(df_3['sick_blood_pressure_diastolic_3'].values.reshape(-1, 1))
df_4['sick_blood_pressure_diastolic_4'] =  s_b_p_d_scaler.transform(df_4['sick_blood_pressure_diastolic_4'].values.reshape(-1, 1))
df_5['sick_blood_pressure_diastolic_5'] =  s_b_p_d_scaler.transform(df_5['sick_blood_pressure_diastolic_5'].values.reshape(-1, 1))
df_6['sick_blood_pressure_diastolic_6'] =  s_b_p_d_scaler.transform(df_6['sick_blood_pressure_diastolic_6'].values.reshape(-1, 1))

s_h_r_scaler = StandardScaler()
s_h_r_scaler.fit(np.array(s_h_r_scaling))
df_0['sick_heart_rhythm_0'] =  s_h_r_scaler.transform(df_0['sick_heart_rhythm_0'].values.reshape(-1, 1))
df_1['sick_heart_rhythm_1'] =  s_h_r_scaler.transform(df_1['sick_heart_rhythm_1'].values.reshape(-1, 1))
df_2['sick_heart_rhythm_2'] =  s_h_r_scaler.transform(df_2['sick_heart_rhythm_2'].values.reshape(-1, 1))
df_3['sick_heart_rhythm_3'] =  s_h_r_scaler.transform(df_3['sick_heart_rhythm_3'].values.reshape(-1, 1))
df_4['sick_heart_rhythm_4'] =  s_h_r_scaler.transform(df_4['sick_heart_rhythm_4'].values.reshape(-1, 1))
df_5['sick_heart_rhythm_5'] =  s_h_r_scaler.transform(df_5['sick_heart_rhythm_5'].values.reshape(-1, 1))
df_6['sick_heart_rhythm_6'] =  s_h_r_scaler.transform(df_6['sick_heart_rhythm_6'].values.reshape(-1, 1))

age_scaler = StandardScaler()
age_scaler.fit(df_0['sick_age_0'].values.reshape(-1, 1))
df_0['sick_age_0'] = age_scaler.transform(df_0['sick_age_0'].values.reshape(-1, 1))
df_1['sick_age_1'] = age_scaler.transform(df_1['sick_age_1'].values.reshape(-1, 1))
df_2['sick_age_2'] = age_scaler.transform(df_2['sick_age_2'].values.reshape(-1, 1))
df_3['sick_age_3'] = age_scaler.transform(df_3['sick_age_3'].values.reshape(-1, 1))
df_4['sick_age_4'] = age_scaler.transform(df_4['sick_age_4'].values.reshape(-1, 1))
df_5['sick_age_5'] = age_scaler.transform(df_5['sick_age_5'].values.reshape(-1, 1))
df_6['sick_age_6'] = age_scaler.transform(df_6['sick_age_6'].values.reshape(-1, 1))

height_scaler = StandardScaler()
height_scaler.fit(df_0['sick_height_0'].values.reshape(-1, 1))
df_0['sick_height_0'] = height_scaler.transform(df_0['sick_height_0'].values.reshape(-1, 1))
df_1['sick_height_1'] = height_scaler.transform(df_1['sick_height_1'].values.reshape(-1, 1))
df_2['sick_height_2'] = height_scaler.transform(df_2['sick_height_2'].values.reshape(-1, 1))
df_3['sick_height_3'] = height_scaler.transform(df_3['sick_height_3'].values.reshape(-1, 1))
df_4['sick_height_4'] = height_scaler.transform(df_4['sick_height_4'].values.reshape(-1, 1))
df_5['sick_height_5'] = height_scaler.transform(df_5['sick_height_5'].values.reshape(-1, 1))
df_6['sick_height_6'] = height_scaler.transform(df_6['sick_height_6'].values.reshape(-1, 1))

weight_scaler = StandardScaler()
weight_scaler.fit(df_0['sick_weight_0'].values.reshape(-1, 1))
df_0['sick_weight_0'] = weight_scaler.transform(df_0['sick_weight_0'].values.reshape(-1, 1))
df_1['sick_weight_1'] = weight_scaler.transform(df_1['sick_weight_1'].values.reshape(-1, 1))
df_2['sick_weight_2'] = weight_scaler.transform(df_2['sick_weight_2'].values.reshape(-1, 1))
df_3['sick_weight_3'] = weight_scaler.transform(df_3['sick_weight_3'].values.reshape(-1, 1))
df_4['sick_weight_4'] = weight_scaler.transform(df_4['sick_weight_4'].values.reshape(-1, 1))
df_5['sick_weight_5'] = weight_scaler.transform(df_5['sick_weight_5'].values.reshape(-1, 1))
df_6['sick_weight_6'] = weight_scaler.transform(df_6['sick_weight_6'].values.reshape(-1, 1))

### Determining their features and columns and removing inefficient features, as well as determining the label or output of the model and dividing the data into training and test data.

We do this in such a way that the constant columns in the 7 stages of induction, such as weight, gender, age and height, are the same for all the x's of our training, so that the model does not observe all the x's and does not suffer from overfitting, as well as the existence of different values ​​of these Don't confuse the 4 columns of our model, separate these 4 mentioned columns from the rest of the x's except the zero x, and after separating the test and training function, connect these 4 columns from x0 to the other x's Let it be the same for all 7 steps.

Also, our labels or y's are all one and fixed, so we consider the labels as one

In [11]:
X0 = df_0.drop(columns=['sick_number_0','estimated_dose_of_propofol_0','actual_dose_of_propofol_0'])

X1 = df_1.drop(columns=['sick_number_1','estimated_dose_of_propofol_1','actual_dose_of_propofol_1','sick_gender_1','sick_age_1','sick_height_1','sick_weight_1'])

X2 = df_2.drop(columns=['sick_number_2','estimated_dose_of_propofol_2','actual_dose_of_propofol_2','sick_gender_2','sick_age_2','sick_height_2','sick_weight_2'])

X3 = df_3.drop(columns=['sick_number_3','estimated_dose_of_propofol_3','actual_dose_of_propofol_3','sick_gender_3','sick_age_3','sick_height_3','sick_weight_3'])

X4 = df_4.drop(columns=['sick_number_4','estimated_dose_of_propofol_4','actual_dose_of_propofol_4','sick_gender_4','sick_age_4','sick_height_4','sick_weight_4'])

X5 = df_5.drop(columns=['sick_number_5','estimated_dose_of_propofol_5','actual_dose_of_propofol_5','sick_gender_5','sick_age_5','sick_height_5','sick_weight_5'])

X6 = df_6.drop(columns=['sick_number_6','estimated_dose_of_propofol_6','actual_dose_of_propofol_6','sick_gender_6','sick_age_6','sick_height_6','sick_weight_6'])

y = df_0['estimated_dose_of_propofol_0']

X_train0, X_test0, X_train1, X_test1, X_train2, X_test2, X_train3, X_test3, X_train4, X_test4, X_train5, X_test5, X_train6, X_test6, y_train, y_test = train_test_split(X0 ,X1 ,X2 ,X3 ,X4 ,X5 ,X6 ,y, test_size=0.3, random_state=42)

X_train1 = pd.concat([X_train0['sick_gender_0'],X_train0['sick_age_0'],X_train0['sick_height_0'],X_train0['sick_weight_0'],X_train1],axis=1)
X_train2 = pd.concat([X_train0['sick_gender_0'],X_train0['sick_age_0'],X_train0['sick_height_0'],X_train0['sick_weight_0'],X_train2],axis=1)
X_train3 = pd.concat([X_train0['sick_gender_0'],X_train0['sick_age_0'],X_train0['sick_height_0'],X_train0['sick_weight_0'],X_train3],axis=1)
X_train4 = pd.concat([X_train0['sick_gender_0'],X_train0['sick_age_0'],X_train0['sick_height_0'],X_train0['sick_weight_0'],X_train4],axis=1)
X_train5 = pd.concat([X_train0['sick_gender_0'],X_train0['sick_age_0'],X_train0['sick_height_0'],X_train0['sick_weight_0'],X_train5],axis=1)
X_train6 = pd.concat([X_train0['sick_gender_0'],X_train0['sick_age_0'],X_train0['sick_height_0'],X_train0['sick_weight_0'],X_train6],axis=1)

X_test1 = pd.concat([X_test0['sick_gender_0'],X_test0['sick_age_0'],X_test0['sick_height_0'],X_test0['sick_weight_0'],X_test1],axis=1)
X_test2 = pd.concat([X_test0['sick_gender_0'],X_test0['sick_age_0'],X_test0['sick_height_0'],X_test0['sick_weight_0'],X_test2],axis=1)
X_test3 = pd.concat([X_test0['sick_gender_0'],X_test0['sick_age_0'],X_test0['sick_height_0'],X_test0['sick_weight_0'],X_test3],axis=1)
X_test4 = pd.concat([X_test0['sick_gender_0'],X_test0['sick_age_0'],X_test0['sick_height_0'],X_test0['sick_weight_0'],X_test4],axis=1)
X_test5 = pd.concat([X_test0['sick_gender_0'],X_test0['sick_age_0'],X_test0['sick_height_0'],X_test0['sick_weight_0'],X_test5],axis=1)
X_test6 = pd.concat([X_test0['sick_gender_0'],X_test0['sick_age_0'],X_test0['sick_height_0'],X_test0['sick_weight_0'],X_test6],axis=1)

### Returning the names of the columns to their original names so that the grid search algorithm for predicting simulator values ​​does not make errors

In [12]:
X_train1 = X_train1.rename({'sick_gender_0' : 'sick_gender_1' , 'sick_age_0' : 'sick_age_1' , 'sick_height_0' : 'sick_height_1' , 'sick_weight_0' : 'sick_weight_1' },axis=1)
X_train2 = X_train2.rename({'sick_gender_0' : 'sick_gender_2' , 'sick_age_0' : 'sick_age_2' , 'sick_height_0' : 'sick_height_2' , 'sick_weight_0' : 'sick_weight_2' },axis=1)
X_train3 = X_train3.rename({'sick_gender_0' : 'sick_gender_3' , 'sick_age_0' : 'sick_age_3' , 'sick_height_0' : 'sick_height_3' , 'sick_weight_0' : 'sick_weight_3' },axis=1)
X_train4 = X_train4.rename({'sick_gender_0' : 'sick_gender_4' , 'sick_age_0' : 'sick_age_4' , 'sick_height_0' : 'sick_height_4' , 'sick_weight_0' : 'sick_weight_4' },axis=1)
X_train5 = X_train5.rename({'sick_gender_0' : 'sick_gender_5' , 'sick_age_0' : 'sick_age_5' , 'sick_height_0' : 'sick_height_5' , 'sick_weight_0' : 'sick_weight_5' },axis=1)
X_train6 = X_train6.rename({'sick_gender_0' : 'sick_gender_6' , 'sick_age_0' : 'sick_age_6' , 'sick_height_0' : 'sick_height_6' , 'sick_weight_0' : 'sick_weight_6' },axis=1)

X_test1 = X_test1.rename({'sick_gender_0' : 'sick_gender_1' , 'sick_age_0' : 'sick_age_1' , 'sick_height_0' : 'sick_height_1' , 'sick_weight_0' : 'sick_weight_1' },axis=1)
X_test2 = X_test2.rename({'sick_gender_0' : 'sick_gender_2' , 'sick_age_0' : 'sick_age_2' , 'sick_height_0' : 'sick_height_2' , 'sick_weight_0' : 'sick_weight_2' },axis=1)
X_test3 = X_test3.rename({'sick_gender_0' : 'sick_gender_3' , 'sick_age_0' : 'sick_age_3' , 'sick_height_0' : 'sick_height_3' , 'sick_weight_0' : 'sick_weight_3' },axis=1)
X_test4 = X_test4.rename({'sick_gender_0' : 'sick_gender_4' , 'sick_age_0' : 'sick_age_4' , 'sick_height_0' : 'sick_height_4' , 'sick_weight_0' : 'sick_weight_4' },axis=1)
X_test5 = X_test5.rename({'sick_gender_0' : 'sick_gender_5' , 'sick_age_0' : 'sick_age_5' , 'sick_height_0' : 'sick_height_5' , 'sick_weight_0' : 'sick_weight_5' },axis=1)
X_test6 = X_test6.rename({'sick_gender_0' : 'sick_gender_6' , 'sick_age_0' : 'sick_age_6' , 'sick_height_0' : 'sick_height_6' , 'sick_weight_0' : 'sick_weight_6' },axis=1)


### Determining the random forest regression model for training on the data and using network search to optimize the model parameters and reduce the model error by using all induction steps.

In [ ]:
rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [i for i in range(10,101,5)],
    'max_depth': [i for i in range(40,171,10)],
    'min_samples_split': [i for i in range(2,16,1)]
}

grid_search_0 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')

grid_search_1 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')

grid_search_2 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')

grid_search_3 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')

grid_search_4 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')

grid_search_5 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')

grid_search_6 = GridSearchCV(rf, param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1,error_score='raise')


grid_search_0.fit(X_train0,y_train)
grid_search_1.fit(X_train1,y_train)
grid_search_2.fit(X_train2,y_train)
grid_search_3.fit(X_train3,y_train)
grid_search_4.fit(X_train4,y_train)
grid_search_5.fit(X_train5,y_train)
grid_search_6.fit(X_train6,y_train)

### Obtaining changes in the importance of the patient's age characteristic by the model after gradually facing the new induction stages and drawing its diagram

In [ ]:
gs0_be = grid_search_0.best_estimator_
gs1_be = grid_search_1.best_estimator_
gs2_be = grid_search_2.best_estimator_
gs3_be = grid_search_3.best_estimator_
gs4_be = grid_search_4.best_estimator_
gs5_be = grid_search_5.best_estimator_
gs6_be = grid_search_6.best_estimator_

plt.plot(['i_s_0','i_s_1','i_s_2','i_s_3','i_s_4','i_s_5','i_s_6'],[gs0_be.feature_importances_[1],gs1_be.feature_importances_[1],gs2_be.feature_importances_[1],gs3_be.feature_importances_[1],gs4_be.feature_importances_[1],gs5_be.feature_importances_[1],gs6_be.feature_importances_[1]])
plt.title("age Importance change by see : induction steps")
plt.show()

### Drawing a graph of the importance of features for all stages of induction

In [ ]:
feature_names = ['gender','age','height','weight','bp_slc','bp_dlc','h_rhythm','d_o_p_a']

fig, axes = plt.subplots(3, 3, figsize=(25, 25)) 
axes = axes.ravel() 

axes[0].plot(feature_names,grid_search_0.best_estimator_.feature_importances_)
axes[0].set_title('feature_importances dtep 0 ')
axes[0].grid(True) 
axes[0].legend()

axes[1].plot(feature_names,grid_search_1.best_estimator_.feature_importances_)
axes[1].set_title('feature_importances dtep 1 ')
axes[1].grid(True) 
axes[1].legend()

axes[2].plot(feature_names,grid_search_2.best_estimator_.feature_importances_)
axes[2].set_title('feature_importances dtep 2 ')
axes[2].grid(True) 
axes[2].legend()

axes[3].plot(feature_names,grid_search_3.best_estimator_.feature_importances_)
axes[3].set_title('feature_importances dtep 3 ')
axes[3].grid(True) 
axes[3].legend()

axes[4].plot(feature_names,grid_search_4.best_estimator_.feature_importances_)
axes[4].set_title('feature_importances dtep 4 ')
axes[4].grid(True) 
axes[4].legend()

axes[5].plot(feature_names,grid_search_5.best_estimator_.feature_importances_)
axes[5].set_title('feature_importances dtep 5 ')
axes[5].grid(True) 
axes[5].legend()

axes[7].plot(feature_names,grid_search_6.best_estimator_.feature_importances_)
axes[7].set_title('feature_importances dtep 6 ')
axes[7].grid(True) 
axes[7].legend()
    
fig.delaxes(axes[8])
fig.delaxes(axes[6])

plt.show()


### Obtaining the mean squared error and R2 error for each induction step with a single machine learning algorithm and different optimizers for each induction step.

In [ ]:
y_pred0 = grid_search_0.best_estimator_.predict(X_test0)
MSE0 = mean_squared_error(y_test, y_pred0)
R2_0 = r2_score(y_test, y_pred0)

y_pred1 = grid_search_1.best_estimator_.predict(X_test1)
MSE1 = mean_squared_error(y_test, y_pred1)
R2_1 = r2_score(y_test, y_pred1)

y_pred2 = grid_search_2.best_estimator_.predict(X_test2)
MSE2 = mean_squared_error(y_test, y_pred2)
R2_2 = r2_score(y_test, y_pred2)

y_pred3 = grid_search_3.best_estimator_.predict(X_test3)
MSE3 = mean_squared_error(y_test, y_pred3)
R2_3 = r2_score(y_test, y_pred3)

y_pred4 = grid_search_4.best_estimator_.predict(X_test4)
MSE4 = mean_squared_error(y_test, y_pred4)
R2_4 = r2_score(y_test, y_pred4)

y_pred5 = grid_search_5.best_estimator_.predict(X_test5)
MSE5 = mean_squared_error(y_test, y_pred5)
R2_5 = r2_score(y_test, y_pred5)

y_pred6 = grid_search_6.best_estimator_.predict(X_test6)
MSE6 = mean_squared_error(y_test, y_pred6)
R2_6 = r2_score(y_test, y_pred6)


print("MSE0:", MSE0 , "MSE1:", MSE1 , "MSE2:", MSE2 , "MSE3:", MSE3 , "MSE4:", MSE4 , "MSE5:", MSE5 , "MSE6:", MSE6)
print("R2_0:", R2_0 , "R2_1:", R2_1 , "R2_2:", R2_2 , "R2_3:", R2_3 , "R2_4:", R2_4 , "R2_5:", R2_5 , "R2_6:", R2_6)

### Draw a diagram of the mentioned errors

In [ ]:


MSE = [MSE0, MSE1, MSE2, MSE3, MSE4, MSE5, MSE6]
R2 = [R2_0, R2_1, R2_2, R2_3, R2_4, R2_5, R2_6]

errors = ['MSE', 'R2']
values = [MSE, R2]

fig, axes = plt.subplots(3, 3, figsize=(15, 10))  
axes = axes.ravel()  

for i in range(len(MSE)):
    x = values[0][i]
    y = values[1][i]
    axes[i].scatter(x, y, color='blue', label=f'Error {i}')
    axes[i].set_xlabel(errors[0])
    axes[i].set_ylabel(errors[1])
    axes[i].set_title(f' error chart {i}')
    axes[i].grid(True) 
    axes[i].legend()

fig.delaxes(axes[7])
fig.delaxes(axes[8])

plt.tight_layout()
plt.show()

colors = plt.cm.viridis(np.linspace(0, 1, 7))  
plt.figure(figsize=(8, 6))
for i in range(len(MSE)):
    plt.scatter(values[0][i], values[1][i], color=colors[i], label=f'error {i}', s=50) 

plt.xlabel(errors[0])
plt.ylabel(errors[1])
plt.title(' all errors chart ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

#### According to the nested lists of systolic and diastolic blood pressure, he noticed a special pattern that the first 5 numbers are constant and the next two numbers are different from the previous numbers but are constant relative to each other, and their difference is a chance compared to their comparison. It is ascending or descending with the average, that is, if it is more than the average, it is more likely to be descending, and if the first 5 numbers are less than the average, the next two numbers are most likely to be ascending, so we simulate this distribution with this algorithm.

In [31]:
def blood_p_simulate(n, systolic_min , systolic_max , diastolic_min , diastolic_max) :
    # 0 : ascending
    # 1 : Descending
    syst_mean = (systolic_min + systolic_max) / 2
    diast_mean = (diastolic_min + diastolic_max) / 2
    systs = []
    diasts = []
    
    for i in range(n) : 
        
        syst_list = []
        systs_chance =[]
        a = np.random.randint(systolic_min,systolic_max)
        if a <= np.round(syst_mean) :
            for j in range(8) :
                systs_chance.append(1)
            for j in range(2) :
                systs_chance.append(0)
        elif a >= np.round(syst_mean) :
            for j in range(8) :
                systs_chance.append(0)
            for j in range(2) :
                systs_chance.append(1)
        s_chnc = np.random.choice(systs_chance)
        if s_chnc == 1 :    
            for j in range(5) :
                syst_list.append(a)
            sec_a = np.random.randint(systolic_min,a+1)
            for j in range(2) :
                syst_list.append(sec_a)
        elif s_chnc == 0 :
            for j in range(5) :
                syst_list.append(a)
            sec_a = np.random.randint(a-1,systolic_max)
            for j in range(2) :
                syst_list.append(sec_a)
        systs.append(syst_list)
        
        diast_list = []
        diast_chance =[]
        b = np.random.randint(diastolic_min,diastolic_max)
        if b <= np.round(diast_mean) :
            for j in range(8) :
                diast_chance.append(1)
            for j in range(2) :
                diast_chance.append(0)
        elif b >= np.round(diast_mean) :
            for j in range(8) :
                diast_chance.append(0)
            for j in range(2) :
                diast_chance.append(1)
        d_chnc = np.random.choice(diast_chance)
        if d_chnc == 1 :    
            for j in range(5) :
                diast_list.append(b)
            sec_b = np.random.randint(diastolic_min,b+1)
            for j in range(2) :
                diast_list.append(sec_b)
        elif d_chnc == 0 :
            for j in range(5) :
                diast_list.append(b)
            sec_b = np.random.randint(b-1,diastolic_max)
            for j in range(2) :
                diast_list.append(sec_b)
        diasts.append(diast_list)
        
    return systs,diasts

#### Now we generate the patient's heart rhythm according to the primary data pattern for the simulation operation

In [32]:
the_first_5_data = np.array(base_data['sick_heart_rhythm'])[:,0:5]

the_last_2_data = np.array(base_data['sick_heart_rhythm'])[:,5:7]

the_first_5_min = 350
the_first_5_max = 0
for i in the_first_5_data : 
    for j in i : 
        if j > the_first_5_max :
            the_first_5_max = j
        if j < the_first_5_min :
            the_first_5_min = j
            
the_last_2_min = 350
the_last_2_max = 0
for i in the_last_2_data : 
    for j in i : 
        if j > the_last_2_max :
            the_last_2_max = j
        if j < the_last_2_min :
            the_last_2_min = j

def heart_rhythm_simulate(n) : 
    # 0 : ascending
    # 1 : Descending
    h_rhythms = []
    for i in range(n) :
        h_rhythm_list = []
        h_rhythm_chance =[]
        a = np.random.randint(the_first_5_min,the_first_5_max)
        for j in range(93) :
            h_rhythm_chance.append(1)
        for j in range(7) :
            h_rhythm_chance.append(0)
        s_chnc = np.random.choice(h_rhythm_chance)
        if s_chnc == 1 :    
            for j in range(5) :
                h_rhythm_list.append(a)
            sec_a = np.random.randint(the_last_2_min,a+1)
            for j in range(2) :
                h_rhythm_list.append(sec_a)
        elif s_chnc == 0 :
            for j in range(5) :
                h_rhythm_list.append(a)
            sec_a = np.random.randint(a-1,the_last_2_max)
            for j in range(2) :
                h_rhythm_list.append(sec_a)
        h_rhythms.append(h_rhythm_list)
    return h_rhythms

#### And finally, we simulate the most complex patterned data, i.e. the depth of the patient's anesthesia

In [33]:
the_first_1_data = np.array(base_data['depth_of_patient_anesthesia'])[:,0:1]

the_2_data = np.array(base_data['depth_of_patient_anesthesia'])[:,1:2]

the_3_data = np.array(base_data['depth_of_patient_anesthesia'])[:,2:3]

the_4_data = np.array(base_data['depth_of_patient_anesthesia'])[:,3:4]

the_5_data = np.array(base_data['depth_of_patient_anesthesia'])[:,4:5]

the_6_data = np.array(base_data['depth_of_patient_anesthesia'])[:,5:6]

the_7_data = np.array(base_data['depth_of_patient_anesthesia'])[:,6:7]

the_first_1_min = 350
the_first_1_max = 0

the_2_min = 350

the_3_min = 350

the_4_min = 350

the_5_min = 350

the_6_min = 350

the_7_min = 350

n = 20
for i in the_first_1_data : 
    for j in i : 
        if j > the_first_1_max :
            the_first_1_max = j
        if j < the_first_1_min :
            the_first_1_min = j
            
for i in the_2_data : 
    for j in i : 
        if j < the_2_min :
            the_2_min = j
            
for i in the_3_data : 
    for j in i : 
        if j < the_3_min :
            the_3_min = j
            
for i in the_4_data : 
    for j in i : 
        if j < the_4_min :
            the_4_min = j
            
for i in the_5_data : 
    for j in i : 
        if j < the_5_min :
            the_5_min = j
            
for i in the_6_data : 
    for j in i : 
        if j < the_6_min :
            the_6_min = j
            
for i in the_7_data : 
    for j in i : 
        if j < the_7_min :
            the_7_min = j
def depth_of_patient_anesthesia_simulate(n) : 
    b = 2
    first_numbers = []
    f_chance = []
    for i in range(the_first_1_min,the_first_1_max+1) : 
        for j in range(int(np.round((i - 64)**b))) :
            f_chance.append(i)
    np.random.shuffle(f_chance)
    for i in range(n) :
        first_numbers.append(np.random.choice(f_chance))
    f_chance.clear()


    second_numbers = []
    for i in range(n) : 
        s_chance = []
        for j in range(min(*first_numbers,the_2_min),first_numbers[i]+1) : 
            for k in range(int(np.round((j-70)**b))) :
                s_chance.append(j)
        np.random.shuffle(s_chance)
        second_numbers.append(np.random.choice(s_chance))
    s_chance.clear()
            
    third_numbers = []
    for i in range(n) : 
        t_chance = []
        for j in range(min(*second_numbers,the_3_min),second_numbers[i]+1) : 
            for k in range(int(np.round((j-70)**b))) :
                t_chance.append(j)
        np.random.shuffle(t_chance)
        third_numbers.append(np.random.choice(t_chance))
    t_chance.clear()
            
    fourth_numbers = []
    for i in range(n) : 
        fo_chance = []
        for j in range(min(*third_numbers,the_4_min),third_numbers[i]+1) : 
            for k in range(int(np.round((130-j)**b))) :
                fo_chance.append(j)
        np.random.shuffle(fo_chance)
        fourth_numbers.append(np.random.choice(fo_chance))
    fo_chance.clear()
            
    fifth_numbers = []
    for i in range(n) : 
        fi_chance = []
        for j in range(min(*fourth_numbers,the_5_min),fourth_numbers[i]+1) : 
            for k in range(int(np.round((130-j)**b))) :
                fi_chance.append(j)
        np.random.shuffle(fi_chance)
        fifth_numbers.append(np.random.choice(fi_chance))
    fi_chance.clear()


    sixth_numbers = []
    for i in range(n) : 
        si_chance = []
        for j in range(min(*fifth_numbers,the_6_min),fifth_numbers[i]+1) : 
            for k in range(int(np.round((130-j)**b))) :
                si_chance.append(j)
        np.random.shuffle(si_chance)
        sixth_numbers.append(np.random.choice(si_chance))
    si_chance.clear()
            
    seventh_numbers = []
    for i in range(n) : 
        se_chance = []
        for j in range(min(*sixth_numbers,the_7_min),sixth_numbers[i]+1) : 
            for k in range(int(np.round((130-j)**b))) :
                se_chance.append(j)
        np.random.shuffle(se_chance)
        seventh_numbers.append(np.random.choice(se_chance))
    se_chance.clear()
    main_list = []
    for i in range(n) :
        main_list.append([first_numbers[i],second_numbers[i],third_numbers[i],fourth_numbers[i],fifth_numbers[i],sixth_numbers[i],seventh_numbers[i]])
    return main_list



#### In the following, with the help of the created function, we will simulate the required number of data (here 300) and the normal columns according to the range and distribution and the main data classes and the columns that contain the lists. We will discover their pattern and simulate it

In [34]:

sim_data = {
            'sick_number' : [],
            'sick_gender': [],
            'sick_age': [],
            'sick_height': [],
            'sick_weight': [],
            'sick_blood_pressure_systolic': [],
            'sick_blood_pressure_diastolic': [],
            'sick_heart_rhythm': [],
            'actual_dose_of_propofol': [],
            'estimated_dose_of_propofol':[],
            'induction_steps': [],
            'depth_of_patient_anesthesia': []
        }

def simulate_data(original_df, n=300):
    dopas = depth_of_patient_anesthesia_simulate(n)
    for i in range(n):
        sim_syst,sim_diast = blood_p_simulate(1,syst_min,syst_max,diast_min,diast_max)
        sim_rhythm = heart_rhythm_simulate(1)
        sim_data['sick_number'].append(i+1)
        sim_data['sick_gender'].append(np.random.choice(original_df['sick_gender'])),
        sim_data['sick_age'].append(np.random.randint(original_df['sick_age'].value_counts().keys().min(),original_df['sick_age'].value_counts().keys().max())),
        sim_data['sick_height'].append(np.random.randint(original_df['sick_height'].value_counts().keys().min(),original_df['sick_height'].value_counts().keys().max())),
        sim_data['sick_weight'].append(np.random.randint(original_df['sick_weight'].value_counts().keys().min(),original_df['sick_weight'].value_counts().keys().max()))
        sim_data['sick_blood_pressure_systolic'].append(sim_syst[0]),
        sim_data['sick_blood_pressure_diastolic'].append(sim_diast[0]),
        sim_data['sick_heart_rhythm'].append(sim_rhythm[0]),
        sim_data['actual_dose_of_propofol'].append(np.random.choice(original_df['actual_dose_of_propofol'])),
        sim_data['estimated_dose_of_propofol'].append(None),
        sim_data['induction_steps'].append([i for i in range(7)]),
        sim_data['depth_of_patient_anesthesia'].append(dopas[i])
        
    return pd.DataFrame(sim_data),sim_data

simulated_df,simulated_data = simulate_data(main_df)

In [ ]:
simulated_df.describe()

### This section is the same as the section above for the main data that was explained earlier

In [36]:
induction_steps_simulated_0 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][0] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][0] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][0] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][0] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}



induction_steps_simulated_1 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][1] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][1] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][1] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][1] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}



induction_steps_simulated_2 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][2] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][2] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][2] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][2] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}



induction_steps_simulated_3 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][3] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][3] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][3] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][3] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}



induction_steps_simulated_4 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][4] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][4] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][4] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][4] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}



induction_steps_simulated_5 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][5] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][5] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][5] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][5] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}



induction_steps_simulated_6 = {'sick_number': [i for i in simulated_data['sick_number']],
            'sick_gender': [i for i in simulated_data['sick_gender']],
            'sick_age': [i for i in simulated_data['sick_age']],
            'sick_height': [i for i in simulated_data['sick_height']],
            'sick_weight': [i for i in simulated_data['sick_weight']],
            'sick_blood_pressure_systolic': [simulated_data['sick_blood_pressure_systolic'][i][6] for i in range(len(simulated_data['sick_blood_pressure_systolic']))],
            'sick_blood_pressure_diastolic': [simulated_data['sick_blood_pressure_diastolic'][i][6] for i in range(len(simulated_data['sick_blood_pressure_diastolic']))],
            'sick_heart_rhythm':[simulated_data['sick_heart_rhythm'][i][6] for i in range(len(simulated_data['sick_heart_rhythm']))],
            'actual_dose_of_propofol':[i for i in(simulated_data['actual_dose_of_propofol'])],
            'estimated_dose_of_propofol':[i for i in(simulated_data['estimated_dose_of_propofol'])],
            'depth_of_patient_anesthesia':[simulated_data['depth_of_patient_anesthesia'][i][6] for i in range(len(simulated_data['depth_of_patient_anesthesia']))]}

### This section is the same as the section above for the main data that was explained earlier

In [37]:
df_simulated_0 = pd.DataFrame()
df_simulated_1 = pd.DataFrame()
df_simulated_2 = pd.DataFrame()
df_simulated_3 = pd.DataFrame()
df_simulated_4 = pd.DataFrame()
df_simulated_5 = pd.DataFrame()
df_simulated_6 = pd.DataFrame()

for key, value in induction_steps_simulated_0.items():
    df_simulated_0[key + "_0"] = value
    
for key, value in induction_steps_simulated_1.items():
    df_simulated_1[key + "_1"] = value
    
for key, value in induction_steps_simulated_2.items():
    df_simulated_2[key + "_2"] = value
    
for key, value in induction_steps_simulated_3.items():
    df_simulated_3[key + "_3"] = value
    
for key, value in induction_steps_simulated_4.items():
    df_simulated_4[key + "_4"] = value
    
for key, value in induction_steps_simulated_5.items():
    df_simulated_5[key + "_5"] = value
    
for key, value in induction_steps_simulated_6.items():
    df_simulated_6[key + "_6"] = value

In [ ]:
df_simulated_0.describe()

### This section is the same as the section above for the main data that was explained earlier

In [39]:
d_o_p_a_scaling_simulated = []
for i in simulated_data['depth_of_patient_anesthesia'] : 
    for j in i : 
        d_o_p_a_scaling_simulated.append([j])
       
s_b_p_s_scaling_simulated = []
for i in simulated_data['sick_blood_pressure_systolic'] :
    for j in i : 
        s_b_p_s_scaling_simulated.append([j])
        
s_b_p_d_scaling_simulated = []
for i in simulated_data['sick_blood_pressure_diastolic'] :
    for j in i : 
        s_b_p_d_scaling_simulated.append([j])
        
s_h_r_scaling_simulated = []
for i in simulated_data['sick_heart_rhythm'] : 
    for j in i : 
        s_h_r_scaling_simulated.append([j])
        
d_o_p_a_scaler_simulated = StandardScaler()
d_o_p_a_scaler_simulated.fit(np.array(d_o_p_a_scaling_simulated))
df_simulated_0['depth_of_patient_anesthesia_0'] =  d_o_p_a_scaler_simulated.transform(df_simulated_0['depth_of_patient_anesthesia_0'].values.reshape(-1, 1))
df_simulated_1['depth_of_patient_anesthesia_1'] =  d_o_p_a_scaler_simulated.transform(df_simulated_1['depth_of_patient_anesthesia_1'].values.reshape(-1, 1))
df_simulated_2['depth_of_patient_anesthesia_2'] =  d_o_p_a_scaler_simulated.transform(df_simulated_2['depth_of_patient_anesthesia_2'].values.reshape(-1, 1))
df_simulated_3['depth_of_patient_anesthesia_3'] =  d_o_p_a_scaler_simulated.transform(df_simulated_3['depth_of_patient_anesthesia_3'].values.reshape(-1, 1))
df_simulated_4['depth_of_patient_anesthesia_4'] =  d_o_p_a_scaler_simulated.transform(df_simulated_4['depth_of_patient_anesthesia_4'].values.reshape(-1, 1))
df_simulated_5['depth_of_patient_anesthesia_5'] =  d_o_p_a_scaler_simulated.transform(df_simulated_5['depth_of_patient_anesthesia_5'].values.reshape(-1, 1))
df_simulated_6['depth_of_patient_anesthesia_6'] =  d_o_p_a_scaler_simulated.transform(df_simulated_6['depth_of_patient_anesthesia_6'].values.reshape(-1, 1))

s_b_p_s_scaler_simulated = StandardScaler()
s_b_p_s_scaler_simulated.fit(np.array(s_b_p_s_scaling_simulated))
df_simulated_0['sick_blood_pressure_systolic_0'] =  s_b_p_s_scaler_simulated.transform(df_simulated_0['sick_blood_pressure_systolic_0'].values.reshape(-1, 1))
df_simulated_1['sick_blood_pressure_systolic_1'] =  s_b_p_s_scaler_simulated.transform(df_simulated_1['sick_blood_pressure_systolic_1'].values.reshape(-1, 1))
df_simulated_2['sick_blood_pressure_systolic_2'] =  s_b_p_s_scaler_simulated.transform(df_simulated_2['sick_blood_pressure_systolic_2'].values.reshape(-1, 1))
df_simulated_3['sick_blood_pressure_systolic_3'] =  s_b_p_s_scaler_simulated.transform(df_simulated_3['sick_blood_pressure_systolic_3'].values.reshape(-1, 1))
df_simulated_4['sick_blood_pressure_systolic_4'] =  s_b_p_s_scaler_simulated.transform(df_simulated_4['sick_blood_pressure_systolic_4'].values.reshape(-1, 1))
df_simulated_5['sick_blood_pressure_systolic_5'] =  s_b_p_s_scaler_simulated.transform(df_simulated_5['sick_blood_pressure_systolic_5'].values.reshape(-1, 1))
df_simulated_6['sick_blood_pressure_systolic_6'] =  s_b_p_s_scaler_simulated.transform(df_simulated_6['sick_blood_pressure_systolic_6'].values.reshape(-1, 1))

s_b_p_d_scaler_simulated = StandardScaler()
s_b_p_d_scaler_simulated.fit(np.array(s_b_p_d_scaling))
df_simulated_0['sick_blood_pressure_diastolic_0'] =  s_b_p_d_scaler_simulated.transform(df_simulated_0['sick_blood_pressure_diastolic_0'].values.reshape(-1, 1))
df_simulated_1['sick_blood_pressure_diastolic_1'] =  s_b_p_d_scaler_simulated.transform(df_simulated_1['sick_blood_pressure_diastolic_1'].values.reshape(-1, 1))
df_simulated_2['sick_blood_pressure_diastolic_2'] =  s_b_p_d_scaler_simulated.transform(df_simulated_2['sick_blood_pressure_diastolic_2'].values.reshape(-1, 1))
df_simulated_3['sick_blood_pressure_diastolic_3'] =  s_b_p_d_scaler_simulated.transform(df_simulated_3['sick_blood_pressure_diastolic_3'].values.reshape(-1, 1))
df_simulated_4['sick_blood_pressure_diastolic_4'] =  s_b_p_d_scaler_simulated.transform(df_simulated_4['sick_blood_pressure_diastolic_4'].values.reshape(-1, 1))
df_simulated_5['sick_blood_pressure_diastolic_5'] =  s_b_p_d_scaler_simulated.transform(df_simulated_5['sick_blood_pressure_diastolic_5'].values.reshape(-1, 1))
df_simulated_6['sick_blood_pressure_diastolic_6'] =  s_b_p_d_scaler_simulated.transform(df_simulated_6['sick_blood_pressure_diastolic_6'].values.reshape(-1, 1))

s_h_r_scaler_simulated = StandardScaler()
s_h_r_scaler_simulated.fit(np.array(s_h_r_scaling_simulated))
df_simulated_0['sick_heart_rhythm_0'] =  s_h_r_scaler_simulated.transform(df_simulated_0['sick_heart_rhythm_0'].values.reshape(-1, 1))
df_simulated_1['sick_heart_rhythm_1'] =  s_h_r_scaler_simulated.transform(df_simulated_1['sick_heart_rhythm_1'].values.reshape(-1, 1))
df_simulated_2['sick_heart_rhythm_2'] =  s_h_r_scaler_simulated.transform(df_simulated_2['sick_heart_rhythm_2'].values.reshape(-1, 1))
df_simulated_3['sick_heart_rhythm_3'] =  s_h_r_scaler_simulated.transform(df_simulated_3['sick_heart_rhythm_3'].values.reshape(-1, 1))
df_simulated_4['sick_heart_rhythm_4'] =  s_h_r_scaler_simulated.transform(df_simulated_4['sick_heart_rhythm_4'].values.reshape(-1, 1))
df_simulated_5['sick_heart_rhythm_5'] =  s_h_r_scaler_simulated.transform(df_simulated_5['sick_heart_rhythm_5'].values.reshape(-1, 1))
df_simulated_6['sick_heart_rhythm_6'] =  s_h_r_scaler_simulated.transform(df_simulated_6['sick_heart_rhythm_6'].values.reshape(-1, 1))

age_scaler_simulated = StandardScaler()
age_scaler_simulated.fit(df_simulated_0['sick_age_0'].values.reshape(-1, 1))
df_simulated_0['sick_age_0'] = age_scaler_simulated.transform(df_simulated_0['sick_age_0'].values.reshape(-1, 1))
df_simulated_1['sick_age_1'] = age_scaler_simulated.transform(df_simulated_1['sick_age_1'].values.reshape(-1, 1))
df_simulated_2['sick_age_2'] = age_scaler_simulated.transform(df_simulated_2['sick_age_2'].values.reshape(-1, 1))
df_simulated_3['sick_age_3'] = age_scaler_simulated.transform(df_simulated_3['sick_age_3'].values.reshape(-1, 1))
df_simulated_4['sick_age_4'] = age_scaler_simulated.transform(df_simulated_4['sick_age_4'].values.reshape(-1, 1))
df_simulated_5['sick_age_5'] = age_scaler_simulated.transform(df_simulated_5['sick_age_5'].values.reshape(-1, 1))
df_simulated_6['sick_age_6'] = age_scaler_simulated.transform(df_simulated_6['sick_age_6'].values.reshape(-1, 1))

height_scaler_simulated = StandardScaler()
height_scaler_simulated.fit(df_simulated_0['sick_height_0'].values.reshape(-1, 1))
df_simulated_0['sick_height_0'] = height_scaler_simulated.transform(df_simulated_0['sick_height_0'].values.reshape(-1, 1))
df_simulated_1['sick_height_1'] = height_scaler_simulated.transform(df_simulated_1['sick_height_1'].values.reshape(-1, 1))
df_simulated_2['sick_height_2'] = height_scaler_simulated.transform(df_simulated_2['sick_height_2'].values.reshape(-1, 1))
df_simulated_3['sick_height_3'] = height_scaler_simulated.transform(df_simulated_3['sick_height_3'].values.reshape(-1, 1))
df_simulated_4['sick_height_4'] = height_scaler_simulated.transform(df_simulated_4['sick_height_4'].values.reshape(-1, 1))
df_simulated_5['sick_height_5'] = height_scaler_simulated.transform(df_simulated_5['sick_height_5'].values.reshape(-1, 1))
df_simulated_6['sick_height_6'] = height_scaler_simulated.transform(df_simulated_6['sick_height_6'].values.reshape(-1, 1))

weight_scaler_simulated = StandardScaler()
weight_scaler_simulated.fit(df_simulated_0['sick_weight_0'].values.reshape(-1, 1))
df_simulated_0['sick_weight_0'] = weight_scaler_simulated.transform(df_simulated_0['sick_weight_0'].values.reshape(-1, 1))
df_simulated_1['sick_weight_1'] = weight_scaler_simulated.transform(df_simulated_1['sick_weight_1'].values.reshape(-1, 1))
df_simulated_2['sick_weight_2'] = weight_scaler_simulated.transform(df_simulated_2['sick_weight_2'].values.reshape(-1, 1))
df_simulated_3['sick_weight_3'] = weight_scaler_simulated.transform(df_simulated_3['sick_weight_3'].values.reshape(-1, 1))
df_simulated_4['sick_weight_4'] = weight_scaler_simulated.transform(df_simulated_4['sick_weight_4'].values.reshape(-1, 1))
df_simulated_5['sick_weight_5'] = weight_scaler_simulated.transform(df_simulated_5['sick_weight_5'].values.reshape(-1, 1))
df_simulated_6['sick_weight_6'] = weight_scaler_simulated.transform(df_simulated_6['sick_weight_6'].values.reshape(-1, 1))

### We have already built and trained the model, so now it's time to predict the model on the new data to get the estimated drug dose, so there is no need to segment the data and prepare it as in the previous step.

In [40]:
XS0 = df_simulated_0.drop(columns=['sick_number_0','estimated_dose_of_propofol_0','actual_dose_of_propofol_0'])

XS1 = df_simulated_1.drop(columns=['sick_number_1','estimated_dose_of_propofol_1','actual_dose_of_propofol_1'])

XS2 = df_simulated_2.drop(columns=['sick_number_2','estimated_dose_of_propofol_2','actual_dose_of_propofol_2'])

XS3 = df_simulated_3.drop(columns=['sick_number_3','estimated_dose_of_propofol_3','actual_dose_of_propofol_3'])

XS4 = df_simulated_4.drop(columns=['sick_number_4','estimated_dose_of_propofol_4','actual_dose_of_propofol_4'])

XS5 = df_simulated_5.drop(columns=['sick_number_5','estimated_dose_of_propofol_5','actual_dose_of_propofol_5'])

XS6 = df_simulated_6.drop(columns=['sick_number_6','estimated_dose_of_propofol_6','actual_dose_of_propofol_6'])

### Using the best round and parameters determined by the network search algorithm according to the induction stage for better prediction

In [41]:
gs0_be = grid_search_0.best_estimator_
gs1_be = grid_search_1.best_estimator_
gs2_be = grid_search_2.best_estimator_
gs3_be = grid_search_3.best_estimator_
gs4_be = grid_search_4.best_estimator_
gs5_be = grid_search_5.best_estimator_
gs6_be = grid_search_6.best_estimator_

### First, the prediction model performs its own analysis on the separated data based on induction stages, then the average of these 7 stages is considered and added to the simulated data as the estimated drug dose.

In [42]:
sim_pred_0 = gs0_be.predict(XS0)
sim_pred_1 = gs1_be.predict(XS1)
sim_pred_2 = gs2_be.predict(XS2)
sim_pred_3 = gs3_be.predict(XS3)
sim_pred_4 = gs4_be.predict(XS4)
sim_pred_5 = gs5_be.predict(XS5)
sim_pred_6 = gs6_be.predict(XS6)

simulated_prediction = [sim_pred_0,sim_pred_1,sim_pred_2,sim_pred_3,sim_pred_4,sim_pred_5,sim_pred_6]
mean_simulated_prediction = []

for j in range(300) : 
    num = np.mean([simulated_prediction[0][j],simulated_prediction[1][j],simulated_prediction[2][j],simulated_prediction[3][j],simulated_prediction[4][j],simulated_prediction[5][j],simulated_prediction[6][j]])
    num*=100
    num//=1
    num/=100
    mean_simulated_prediction.append(num)
    
sim_data['estimated_dose_of_propofol'] = mean_simulated_prediction

### Converting the completed data into a dataframe and describing it

In [ ]:
simulate_prediction_data = pd.DataFrame(sim_data)
simulate_prediction_data.describe()